# 🔄 DataSens - Collecte Journalière Automatisée

**Objectif** : Orchestrer la collecte quotidienne des 5 sources de données en production

**Stratégie d'enrichissement continu** :
- ⏰ Exécution planifiée (CRON, Prefect, Airflow)
- 🔒 Déduplication automatique (hash SHA256)
- 📊 Monitoring et logs de collecte
- 🔄 Reprise sur erreur (retry automatique)
- 💾 Backup PostgreSQL quotidien

**Sources collectées** :
1. Kaggle CSV (hebdomadaire)
2. OpenWeatherMap API (4x/jour)
3. RSS Multi-Sources (toutes les heures)
4. Web Scraping (quotidien)
5. GDELT Big Data (toutes les 15 min)

## 📦 Imports et Configuration

In [10]:
import os
import sys
from pathlib import Path
import datetime as dt
import pandas as pd
import subprocess
from dotenv import load_dotenv

# Environnement
ROOT = Path.cwd().parent
load_dotenv(ROOT / ".env")

# PostgreSQL
PG_USER = os.getenv("POSTGRES_USER", "ds_user")
PG_PASS = os.getenv("POSTGRES_PASSWORD", "ds_pass")
PG_HOST = os.getenv("POSTGRES_HOST", "localhost")
PG_PORT = int(os.getenv("POSTGRES_PORT", 5432))
PG_DB = os.getenv("POSTGRES_DB", "datasens")

print("✅ Configuration chargée")
print(f"📂 ROOT : {ROOT}")
print(f"🗄️  PostgreSQL : {PG_USER}@{PG_HOST}:{PG_PORT}/{PG_DB}")

✅ Configuration chargée
📂 ROOT : c:\Users\Utilisateur\Desktop\Datasens_Project
🗄️  PostgreSQL : ds_user@localhost:5432/datasens


## 📝 Système de versioning et logs

Traçabilité complète de chaque collecte journalière

In [17]:
VERSION_FILE = ROOT / "README_VERSIONNING.md"
VERSIONS_DIR = ROOT / "datasens" / "versions"
VERSIONS_DIR.mkdir(parents=True, exist_ok=True)

def log_version(action: str, details: str = ""):
    """Logger simple : timestamp + action + détails → README_VERSIONNING.md"""
    now = dt.datetime.now(dt.UTC).strftime("%Y-%m-%d %H:%M:%S")
    entry = f"- **{now} UTC** | `{action}` | {details}\n"
    
    with open(VERSION_FILE, "a", encoding="utf-8") as f:
        f.write(entry)
    
    print(f"📝 Log : {action} — {details}")

def save_postgres_snapshot(note="Snapshot PostgreSQL quotidien"):
    """Crée un dump PostgreSQL horodaté via Docker"""
    timestamp = dt.datetime.now(dt.UTC).strftime("%Y%m%d_%H%M%S")
    dump_name = f"datasens_pg_v{timestamp}.sql"
    dump_path = VERSIONS_DIR / dump_name
    
    # Détecter automatiquement le conteneur PostgreSQL
    try:
        result_ps = subprocess.run(
            ["docker", "ps", "--filter", "name=postgres", "--format", "{{.Names}}"],
            capture_output=True, text=True, check=True
        )
        container_name = result_ps.stdout.strip().split('\n')[0]
        
        if not container_name:
            raise Exception("Aucun conteneur PostgreSQL trouvé")
        
        print(f"🐳 Conteneur détecté : {container_name}")
    except Exception as e:
        print(f"⚠️ Impossible de détecter le conteneur : {e}")
        print("   Tentative avec nom par défaut 'datasens-postgres'")
        container_name = "datasens-postgres"
    
    # Utiliser Docker pour pg_dump
    cmd = [
        "docker", "exec",
        container_name,
        "pg_dump",
        "-U", PG_USER,
        PG_DB
    ]
    
    try:
        # Exécuter la commande avec encodage UTF-8 pour gérer les caractères français
        result = subprocess.run(cmd, check=True, capture_output=True, text=True, encoding='utf-8', errors='replace')
        
        # Écrire le dump dans le fichier avec encodage UTF-8
        with open(dump_path, "w", encoding="utf-8") as f:
            f.write(result.stdout)
        
        log_version("PG_SNAPSHOT", f"{dump_name} — {note}")
        print(f"✅ Snapshot PostgreSQL créé : {dump_name}")
        print(f"   Taille : {dump_path.stat().st_size / 1024 / 1024:.2f} MB")
        return dump_path
    except FileNotFoundError:
        print("⚠️ Docker non trouvé. Assurez-vous que Docker Desktop est démarré.")
        log_version("PG_SNAPSHOT_FAIL", "Docker manquant ou non démarré")
        return None
    except subprocess.CalledProcessError as e:
        print(f"❌ Erreur pg_dump via Docker : {e.stderr}")
        print(f"   Conteneur utilisé : {container_name}")
        print("   Vérifiez que le conteneur PostgreSQL est running avec : docker ps")
        log_version("PG_SNAPSHOT_ERROR", str(e.stderr)[:100])
        return None

# Initialiser le fichier de versioning s'il n'existe pas
if not VERSION_FILE.exists():
    with open(VERSION_FILE, "w", encoding="utf-8") as f:
        f.write("# 📘 Historique des versions DataSens\n\n")
    print(f"✅ Fichier de versioning créé : {VERSION_FILE}")

log_version("COLLECTE_JOURNALIERE_INIT", "Démarrage script collecte quotidienne")
print("\n🔧 Fonctions de versioning chargées")

📝 Log : COLLECTE_JOURNALIERE_INIT — Démarrage script collecte quotidienne

🔧 Fonctions de versioning chargées


## 📊 Statistiques pré-collecte

État de la base avant la collecte journalière

In [13]:
from sqlalchemy import create_engine, text

PG_URL = f"postgresql+psycopg2://{PG_USER}:{PG_PASS}@{PG_HOST}:{PG_PORT}/{PG_DB}"
engine = create_engine(PG_URL, future=True)

with engine.connect() as conn:
    # Total documents
    total_docs = conn.execute(text("SELECT COUNT(*) FROM document")).scalar()
    
    # Par source (uniquement les sources actives avec documents)
    query_sources = text("""
        SELECT s.nom, COUNT(d.id_doc) as nb_docs
        FROM source s
        LEFT JOIN flux f ON s.id_source = f.id_source
        LEFT JOIN document d ON f.id_flux = d.id_flux
        GROUP BY s.id_source, s.nom
        HAVING COUNT(d.id_doc) > 0
        ORDER BY nb_docs DESC
    """)
    df_sources = pd.read_sql_query(query_sources, conn)

print(f"📊 ÉTAT PRÉ-COLLECTE")
print(f"{'='*60}")
print(f"Total documents : {total_docs:,}")
print(f"\nRépartition par source (sources actives uniquement) :")
print(df_sources.to_string(index=False))
print(f"\n📈 {len(df_sources)} sources actives | {total_docs:,} documents au total")
print(f"⏰ Timestamp : {dt.datetime.now(dt.UTC).isoformat()}Z")

📊 ÉTAT PRÉ-COLLECTE
Total documents : 25,047

Répartition par source (sources actives uniquement) :
                                                        nom  nb_docs
                                                 Kaggle CSV    24683
                                 Web Scraping Multi-Sources      265
Flux RSS Multi-Sources (Franceinfo + 20 Minutes + Le Monde)       99

📈 3 sources actives | 25,047 documents au total
⏰ Timestamp : 2025-10-28T13:49:16.265701+00:00Z


## 🔄 Planification des collectes

### Stratégie de fréquence par source

| Source | Fréquence | Justification |
|--------|-----------|---------------|
| **Kaggle CSV** | Hebdomadaire | Datasets statiques peu mis à jour |
| **OpenWeatherMap** | 4x/jour (6h, 12h, 18h, 00h) | Météo temps réel, 4 relevés quotidiens suffisants |
| **RSS Multi-Sources** | Toutes les heures | Actualités changeantes, rythme médiatique |
| **Web Scraping** | Quotidien (2h du matin) | Éviter surcharge serveurs, respect rate limits |
| **GDELT Big Data** | Toutes les 15 min | Flux temps réel, events mondiaux |

### Implémentation avec Prefect (recommandé)

```python
# Exemple avec Prefect pour orchestration
from prefect import flow, task
from prefect.schedules import CronSchedule

@task(retries=3, retry_delay_seconds=300)
def collect_rss():
    # Code collecte RSS
    pass

@task(retries=3)
def collect_owm():
    # Code collecte OpenWeatherMap
    pass

@flow(name="datasens-daily-collection")
def daily_collection_flow():
    collect_rss()
    collect_owm()
    # ... autres sources

# Planification CRON
# RSS : 0 * * * * (toutes les heures)
# OWM : 0 6,12,18,0 * * * (6h, 12h, 18h, 00h)
# Scraping : 0 2 * * * (2h du matin)
```

### Alternative : Airflow DAG

```python
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import timedelta

default_args = {
    'owner': 'datasens',
    'retries': 3,
    'retry_delay': timedelta(minutes=5)
}

dag = DAG(
    'datasens_daily_collection',
    default_args=default_args,
    schedule_interval='0 2 * * *',  # 2h du matin
    catchup=False
)

task_rss = PythonOperator(
    task_id='collect_rss',
    python_callable=collect_rss_function,
    dag=dag
)
```

## 🎯 Déduplication intelligente

Mécanisme pour éviter les doublons lors des collectes répétées

In [14]:
import hashlib

def sha256(text: str) -> str:
    """Hash SHA256 pour empreinte unique de document"""
    return hashlib.sha256(text.encode('utf-8')).hexdigest()

# La déduplication se fait automatiquement grâce à :
# 1. Colonne hash_fingerprint UNIQUE dans PostgreSQL
# 2. Clause ON CONFLICT DO NOTHING dans insert_documents()

print("✅ Déduplication automatique activée")
print("   Méthode : SHA256 hash_fingerprint + UNIQUE constraint")
print("   Comportement : Les doublons sont silencieusement ignorés (ON CONFLICT DO NOTHING)")

✅ Déduplication automatique activée
   Méthode : SHA256 hash_fingerprint + UNIQUE constraint
   Comportement : Les doublons sont silencieusement ignorés (ON CONFLICT DO NOTHING)


## 📈 Monitoring et alertes

Suivi de la santé des collectes

In [15]:
def check_collection_health():
    """Vérifie que chaque source a collecté des données récemment (< 24h)"""
    
    with engine.connect() as conn:
        query = text("""
            SELECT 
                s.nom,
                COUNT(d.id_doc) as total_docs,
                MAX(f.date_collecte) as derniere_collecte,
                NOW() - MAX(f.date_collecte) as age_dernier_flux
            FROM source s
            LEFT JOIN flux f ON s.id_source = f.id_source
            LEFT JOIN document d ON f.id_flux = d.id_flux
            GROUP BY s.id_source, s.nom
            HAVING COUNT(d.id_doc) > 0 OR MAX(f.date_collecte) IS NOT NULL
            ORDER BY derniere_collecte DESC NULLS LAST
        """)
        df = pd.read_sql_query(query, conn)
    
    print("🏥 SANTÉ DES COLLECTES")
    print("="*80)
    
    sources_actives = 0
    sources_obsoletes = 0
    
    for _, row in df.iterrows():
        nom = row['nom']
        total = row['total_docs']
        derniere = row['derniere_collecte']
        
        if pd.isna(derniere):
            status = "❌ JAMAIS COLLECTÉ"
        elif row['age_dernier_flux'].total_seconds() > 86400:  # > 24h
            status = f"⚠️ OBSOLÈTE ({derniere})"
            sources_obsoletes += 1
        else:
            status = f"✅ OK ({derniere})"
            sources_actives += 1
        
        print(f"{nom:50s} | {total:6,} docs | {status}")
    
    print(f"\n{'='*80}")
    print(f"📊 Résumé : {sources_actives} sources actives (<24h) | {sources_obsoletes} sources obsolètes (>24h)")
    
    # Avertissements si nécessaire
    if sources_obsoletes > 0:
        print(f"⚠️  {sources_obsoletes} source(s) n'ont pas collecté depuis >24h - Vérifier les planifications")
    
    return df

health_df = check_collection_health()

🏥 SANTÉ DES COLLECTES
Web Scraping Multi-Sources                         |    265 docs | ✅ OK (2025-10-28 13:19:43.201839)
Flux RSS Multi-Sources (Franceinfo + 20 Minutes + Le Monde) |     99 docs | ✅ OK (2025-10-28 13:19:13.302066)
OpenWeatherMap                                     |      0 docs | ✅ OK (2025-10-28 13:19:09.552343)
Kaggle CSV                                         | 24,683 docs | ✅ OK (2025-10-28 13:18:30.750655)

📊 Résumé : 4 sources actives (<24h) | 0 sources obsolètes (>24h)


## 💾 Backup automatique quotidien

In [18]:
# Créer le snapshot PostgreSQL quotidien
snapshot_path = save_postgres_snapshot("Backup quotidien automatique")

if snapshot_path:
    print(f"\n✅ Backup PostgreSQL : {snapshot_path}")
    print(f"   Taille : {snapshot_path.stat().st_size / 1024:.2f} Ko")
else:
    print("\n⚠️ Snapshot non créé automatiquement.")
    print("   Commande manuelle (dans le terminal) :")
    print(f"   docker exec datasens-postgres pg_dump -U {PG_USER} {PG_DB} > backup.sql")

🐳 Conteneur détecté : datasens-postgres
📝 Log : PG_SNAPSHOT — datasens_pg_v20251028_135152.sql — Backup quotidien automatique
✅ Snapshot PostgreSQL créé : datasens_pg_v20251028_135152.sql
   Taille : 5.81 MB

✅ Backup PostgreSQL : c:\Users\Utilisateur\Desktop\Datasens_Project\datasens\versions\datasens_pg_v20251028_135152.sql
   Taille : 5949.72 Ko
📝 Log : PG_SNAPSHOT — datasens_pg_v20251028_135152.sql — Backup quotidien automatique
✅ Snapshot PostgreSQL créé : datasens_pg_v20251028_135152.sql
   Taille : 5.81 MB

✅ Backup PostgreSQL : c:\Users\Utilisateur\Desktop\Datasens_Project\datasens\versions\datasens_pg_v20251028_135152.sql
   Taille : 5949.72 Ko


## 📊 Rapport final post-collecte

In [19]:
with engine.connect() as conn:
    # Total documents après collecte
    total_docs_after = conn.execute(text("SELECT COUNT(*) FROM document")).scalar()
    
    # Nouveaux documents (dernières 24h)
    query_new = text("""
        SELECT COUNT(*) 
        FROM document d
        JOIN flux f ON d.id_flux = f.id_flux
        WHERE f.date_collecte >= NOW() - INTERVAL '24 hours'
    """)
    new_docs = conn.execute(query_new).scalar()

print(f"\n📊 RAPPORT POST-COLLECTE")
print(f"{'='*60}")
print(f"Total documents : {total_docs_after:,} (avant: {total_docs:,})")
print(f"Nouveaux documents (24h) : {new_docs:,}")
print(f"Croissance : +{total_docs_after - total_docs:,} documents")
print(f"\n⏰ Fin collecte : {dt.datetime.now(dt.UTC).isoformat()}Z")

log_version("COLLECTE_JOURNALIERE_FIN", f"+{new_docs} nouveaux documents en 24h")


📊 RAPPORT POST-COLLECTE
Total documents : 25,047 (avant: 25,047)
Nouveaux documents (24h) : 25,047
Croissance : +0 documents

⏰ Fin collecte : 2025-10-28T13:52:00.552932+00:00Z
📝 Log : COLLECTE_JOURNALIERE_FIN — +25047 nouveaux documents en 24h


## 🚀 Déploiement en production - GitHub Actions (OPTION RETENUE)

### ✅ Configuration GitHub Actions déployée

Le fichier `.github/workflows/daily-collection.yml` a été créé avec :

**Fonctionnalités** :
- 🕐 Exécution automatique quotidienne à 2h UTC (CRON)
- 🖱️ Exécution manuelle possible (workflow_dispatch)
- 🐳 Services Docker intégrés (PostgreSQL, Redis, MinIO)
- 📊 Génération de rapports HTML automatique
- 💾 Sauvegarde des artifacts pendant 30 jours
- 🚨 Notifications en cas d'échec
- 📈 Extraction automatique des statistiques

**Services démarrés automatiquement** :
- PostgreSQL 15-alpine (port 5432)
- Redis Alpine (port 6379)
- MinIO (ports 9000, 9001)

### 🔐 Secrets GitHub à configurer

Allez dans **Settings → Secrets and variables → Actions → New repository secret** :

```
KAGGLE_USERNAME=votre_username_kaggle
KAGGLE_KEY=votre_cle_api_kaggle
OWM_API_KEY=votre_cle_openweathermap
```

### 📝 Étapes de déploiement

1. **Pousser le code sur GitHub** :
   ```bash
   git add .github/workflows/daily-collection.yml
   git add notebooks/collecte_journaliere.ipynb
   git commit -m "🚀 Add GitHub Actions daily collection workflow"
   git push origin main
   ```

2. **Configurer les secrets** (voir ci-dessus)

3. **Tester l'exécution manuelle** :
   - Aller dans l'onglet **Actions** du repository
   - Sélectionner le workflow "📊 DataSens - Collecte Quotidienne Automatisée"
   - Cliquer sur **Run workflow** → **Run workflow**

4. **Vérifier les artifacts** :
   - Après exécution, télécharger `collection-report-XXX.zip`
   - Contient : notebooks exécutés (.ipynb), rapports HTML, dumps SQL

### 🕐 Planification CRON

```yaml
schedule:
  - cron: '0 2 * * *'  # 2h UTC = 3h Paris (hiver) / 4h Paris (été)
```

**Modifier la fréquence** (si nécessaire) :
- Toutes les 6h : `'0 */6 * * *'`
- Deux fois par jour (2h et 14h) : `'0 2,14 * * *'`
- Toutes les heures : `'0 * * * *'`

### 📊 Monitoring des exécutions

- **Statut temps réel** : Onglet Actions → dernier run
- **Logs détaillés** : Cliquer sur un job → voir les steps
- **Rapports HTML** : Artifacts → télécharger → ouvrir .html
- **Historique** : Actions → filtrer par statut (success/failure)

### 🎯 Avantages GitHub Actions vs autres solutions

| Critère | GitHub Actions | CRON local | Windows Task Scheduler |
|---------|----------------|------------|------------------------|
| **Infrastructure** | ☁️ Cloud gratuit | 💻 Serveur local | 💻 Machine locale |
| **Coût** | Gratuit (2000 min/mois) | Coût serveur | Gratuit |
| **Fiabilité** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐ |
| **Logs** | Interface web | Fichiers logs | Observateur d'événements |
| **Artifacts** | Stockage 30j | Manuel | Manuel |
| **Rapports HTML** | Auto-généré | À coder | À coder |
| **Notifications** | Intégré | À configurer | À configurer |
| **Services Docker** | Intégré | À installer | À installer |